In [55]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex Explainable AI - Feature attributions

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.2

## Installation

Install the following packages required to execute this notebook.

In [56]:
! pip3 install --upgrade --q google-cloud-aiplatform==1.26 google-cloud-storage tabulate
! pip3 install --upgrade --q tensorflow 

### Imports

In [57]:
from google.cloud import aiplatform as vertex_ai

## Setup

In [58]:
PROJECT_ID = "my_project"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [59]:
REGION = "europe-west4"  # @param {type: "string"}

## Run AutoML training pipeline

Next, you run the DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: If `True`, training maybe completed before using the entire budget if the service believes it cannot further improve on the model objective measurements.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto 20 minutes.

In [60]:
# model = dag.run(
#    dataset=dataset,
#    model_display_name="iris",
#    training_fraction_split=0.6,
#    validation_fraction_split=0.2,
#    test_fraction_split=0.2,
#    budget_milli_node_hours=8000,
#    disable_early_stopping=False,
#    target_column=label_column,
#)

**I've already run the training, so I'll just create a model object with the trained model:**

In [61]:
MODEL_ID = '4895582119742406656'
model = vertex_ai.Model(model_name=MODEL_ID)


## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method, with the following parameters:

- `machine_type`: The type of compute machine.

This can take up to 20 minutes.

In [62]:
# endpoint = model.deploy(machine_type="n1-standard-4")

**I've already deployed the model for online predictions, so I'll just create an endpoint object with the deployed endpoint:**

In [63]:
ENDPOINT_ID = '4230951530007625728'
endpoint = vertex_ai.Endpoint(endpoint_name=ENDPOINT_ID)

## Send a online prediction request with explainability

Send a online prediction with explainability to your deployed model. In this method, the predicted response will include an explanation on how the features contributed to the explanation.

### Make test item

You will use synthetic data as a test data item. Don't be concerned that we are using synthetic data -- we just want to demonstrate how to make a prediction.

In [64]:
INSTANCE = {
    "double_field_0": 1.4, # petal_length
    "double_field_1": 1.3, # petal_width
    "double_field_2": 5.1, # sepal_length
    "double_field_3": 2.8, # sepal_width
}

### Make the prediction with explanation

Now that your `Model` resource is deployed to an `Endpoint` resource, one can do online explanations by sending prediction requests to the `Endpoint` resource.

#### Request

The format of each instance is:

    [feature_list]

Since the explain() method can take multiple items (instances), send your single test item as a list of one test item.

#### Response

The response from the explain() call is a Python dictionary with the following entries:

- `ids`: The internal assigned unique identifiers for each prediction request.
- `displayNames`: The class names for each class label.
- `confidences`: For classification, the predicted confidence, between 0 and 1, per class label.
- `values`: For regression, the predicted value.
- `deployed_model_id`: The Vertex AI identifier for the deployed `Model` resource which did the predictions.
- `explanations`: The feature attributions

In [65]:
instances_list = [INSTANCE]

prediction = endpoint.explain(instances_list)
print(prediction)

Prediction(predictions=[{'classes': ['1', '2', '0'], 'scores': [0.3133177161216736, 0.3430679142475128, 0.3436143398284912]}], deployed_model_id='5154248827246477312', model_version_id=None, model_resource_name=None, explanations=[attributions {
  baseline_output_value: 0.3388684093952179
  instance_output_value: 0.3436143398284912
  feature_attributions {
    struct_value {
      fields {
        key: "double_field_0"
        value {
          number_value: 0.003399355337023735
        }
      }
      fields {
        key: "double_field_1"
        value {
          number_value: 0.002479728311300278
        }
      }
      fields {
        key: "double_field_2"
        value {
          number_value: 0.001412386074662209
        }
      }
      fields {
        key: "double_field_3"
        value {
          number_value: -0.002545539289712906
        }
      }
    }
  }
  output_index: 2
  output_display_name: "0"
  approximation_error: 0.003513674562531533
  output_name: "scores"
}


### Understanding the explanations response

First, you will look what your model predicted and compare it to the actual value.

In [66]:
import numpy as np

try:
    label = np.argmax(prediction[0][0]["scores"])
    cls = prediction[0][0]["classes"][label]
    print("Predicted Value:", cls, prediction[0][0]["scores"][label])
except:
    pass

Predicted Value: 0 0.3436143398284912


### Examine feature attributions

Next you will look at the feature attributions for this particular example. Positive attribution values mean a particular feature pushed your model prediction up by that amount, and vice versa for negative attribution values.

In [67]:
from tabulate import tabulate

feature_names = ["double_field_0", "double_field_1", "double_field_2", "double_field_3"] # ["petal_length", "petal_width", "sepal_length", "sepal_width"]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, INSTANCE[val], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

Feature name      Feature value    Attribution value
--------------  ---------------  -------------------
double_field_0              1.4           0.00339936
double_field_1              1.3           0.00247973
double_field_2              5.1           0.00141239
double_field_3              2.8          -0.00254554


### Check your explanations and baselines

To better make sense of the feature attributions you're getting, you should compare them with your model's baseline. In most cases, the sum of your attribution values + the baseline should be very close to your model's predicted value for each input. Also note that for regression models, the `baseline_score` returned from AI Explanations will be the same for each example sent to your model. For classification models, each class will have its own baseline.

In this section you'll send 10 test examples to your model for prediction in order to compare the feature attributions with the baseline. Then you'll run each test example's attributions through a sanity check in the `sanity_check_explanations` method.

#### Get explanations

In [69]:
import random

# Prepare 10 test examples to your model for prediction using a random distribution to generate
# test instances
instances = []
for i in range(10):
    pl = random.uniform(1.0, 2.0)  # petal_length
    pw = random.uniform(1.0, 2.0)  # petal_width
    sl = random.uniform(4.0, 6.0)  # sepal_length
    sw = random.uniform(2.0, 4.0)  # sepal_width

    instances.append(
        {"double_field_0": pl, "double_field_1": pw, "double_field_2": sl, "double_field_3": sw}
    )

response = endpoint.explain(instances)

response

Prediction(predictions=[{'scores': [0.3109892904758453, 0.3446283340454102, 0.3443824052810669], 'classes': ['1', '2', '0']}, {'classes': ['1', '2', '0'], 'scores': [0.3086515665054321, 0.3263656198978424, 0.3649827837944031]}, {'scores': [0.3233571648597717, 0.3353742957115173, 0.3412684798240662], 'classes': ['1', '2', '0']}, {'scores': [0.3126264214515686, 0.3446370661258698, 0.342736542224884], 'classes': ['1', '2', '0']}, {'scores': [0.3224583864212036, 0.3354324996471405, 0.3421091139316559], 'classes': ['1', '2', '0']}, {'scores': [0.3053627610206604, 0.3286619186401367, 0.3659752905368805], 'classes': ['1', '2', '0']}, {'classes': ['1', '2', '0'], 'scores': [0.3120300769805908, 0.3248915374279022, 0.3630782961845398]}, {'scores': [0.3227210938930511, 0.3380762934684753, 0.3392026126384735], 'classes': ['1', '2', '0']}, {'scores': [0.3066827952861786, 0.3246940076351166, 0.3686231076717377], 'classes': ['1', '2', '0']}, {'scores': [0.3037001490592957, 0.3267928659915924, 0.36950

#### Sanity check

In the function below you perform a sanity check on the explanations.

In [70]:
import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

baseline: 0.3310493528842926
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3310493528842926
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
baseline: 0.3388684093952179
You might not get attributions.
0  out of  1  sanity checks passed.
